# TM10007 Group 4 -- ECG Data

Hallo tekst

In [4]:
# Loading packages
import zipfile
import os
import pandas as pd
import sklearn as sk
import numpy as np


## 1. Data loading, cleaning and visualization

Below we load the ECG data, convert it to a dataframe an clean the data.First we are going to start with loading the CS file into a dataframe

In [ ]:
# Load CSV file into dataframe
!git clone https://github.com/JosefienBerg/TM10007_ML_ECG_group4.git

with zipfile.ZipFile('/content/TM10007_ML_ECG_group4/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/TM10007_ML_ECG_group4/ecg')

data = pd.read_csv('/content/TM10007_ML_ECG_group4/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
print(data)

Cloning into 'TM10007_ML_ECG_group4'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 147 (delta 38), reused 22 (delta 22), pack-reused 82
Receiving objects: 100% (147/147), 67.95 MiB | 30.83 MiB/s, done.
Resolving deltas: 100% (62/62), done.
The number of samples: 827
The number of columns: 9001
            0_0         0_1         0_2        0_3        0_4         0_5  \
0     47.685046   48.416904   37.371002  61.604757  58.732738  213.067966   
1    152.726718  297.353726  111.168880  25.419423  14.636085   24.609688   
2      1.601260    3.882169   22.978997  21.673040   8.919484   18.329286   
3      1.388947    3.052483    3.084103   4.627886  10.016196   15.020347   
4      3.625561    3.728466    6.205367  17.722897   5.699401    9.024836   
..          ...         ...         ...        ...        ...         ...   
822   14.673713   13.879148   10.000083   8.337630   3.55730

Now we are going to test if there is data missing in our dataset. We conclude that NaNs are missing data and if an entire row or column has zero's, this is also seen as missing data.

In [ ]:
# Test dataset for missing data
df.isnull()


In [ ]:
# Test dataset for missing rows/columns

In [ ]:
# Cleaning dataset

Visualisatie van de data

In [ ]:
# Code voor ziek en niet ziek

In [ ]:
# Code voor visualisatie

## 2. Creating the experimental setup
Below we split the dataset in test data and training data. Then we further split the trainingdata in order to make different training and validation sets for our different classifiers

In [55]:
# Initial split

# packages 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Defining amount of diseased('1') patients
print(f'Amount of diseased patients: {sum(data["label"])}')

#splitting the whole data set in a train and test set
def train_test_split(x, y, train_size = 0.8, random_state = 0):
  return x_train, x_test, y_train, y_test 

col = "label"
x_train, x_test, y_train, y_test = train_test_split(data.loc[:, data.columns !=col], data.loc[:, "label"].values)
print(x_train)

#splitting the train set in a train and validation set
kf = KFold(n_splits=2)
kf.get_n_splits(x_train)

Amount of diseased patients: 146
             0_0          0_1         0_2         0_3         0_4         0_5  \
593     8.358655    20.946035    9.784189   17.274924   11.670394   35.207370   
780  2244.050479  1067.521450  184.271101   71.748932   47.949082   27.896622   
71   1072.494116    82.493852   46.458215   87.448854  108.303842  110.689626   
49      0.835017     6.575501   12.352972   35.881337   20.712190   49.123780   
267     3.658617     4.205258    6.057089    5.468345    9.699800    9.456077   
..           ...          ...         ...         ...         ...         ...   
763    34.147492    50.031748   81.418305   91.320333   63.807030   62.255815   
192     5.856814     1.322058    4.907688    3.574215    3.317540    7.837192   
629    15.178521     0.769534   27.474748   22.930420   25.603870   26.328295   
559  2920.585238   485.861964   82.289741  124.215917   16.849693   52.066079   
684   372.355214    43.892484   68.014305   52.821319   54.022098   24.14156

2

hallo text

In [ ]:
# Cross-validatie

hallo text

In [ ]:
# Extra blokje

## 3. Feature scaling

Hallo tekst

In [ ]:
# Blokje 1

Hallo tekst

In [ ]:
# Blokje 2

Hallo tekst

In [ ]:
# Blokje 3

## 4. Feature selection

Hallo tekst

In [ ]:
# Blokje 1

Hallo tekst

In [ ]:
# Blokje 2

Hallo tekst

In [ ]:
# Blokje 3